In [128]:
import os
import re
import time
import requests
import pandas as pd
# import anarci  # Uncomment this if you have the ANARCI package installed and configured.

###############################################################################
# Function: fetch_antigen_sequence
###############################################################################
def fetch_antigen_sequence(pdb_code, chain):
    """
    Fetch the antigen FASTA sequence from the RCSB PDB for the specified pdb_code and chain.

    This function retrieves the FASTA file via the RCSB PDB API and parses it to return the
    sequence corresponding to the desired chain. If the FASTA header does not explicitly list
    the desired chain (using the common "Chain" or "Chains" patterns), it will also search for an
    "[auth ...]" designation.

    Parameters:
        pdb_code (str): The PDB identifier (e.g., "1ADQ").
        chain (str): The chain identifier (e.g., "A").

    Returns:
        str: The extracted antigen sequence corresponding to the specified chain.

    Raises:
        ValueError: If an HTTP error occurs or the desired chain is not found in the FASTA file.
    """
    # Build the URL for fetching the FASTA data from RCSB.
    url = f"https://www.rcsb.org/fasta/entry/{pdb_code}?chain={chain}"

    try:
        # Retrieve the FASTA file from the RCSB API.
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses (e.g., 404)
    except requests.exceptions.HTTPError as http_err:
        raise ValueError(f"HTTP error occurred when fetching PDB {pdb_code} chain {chain}: {http_err}")
    except Exception as err:
        raise ValueError(f"An error occurred when fetching PDB {pdb_code} chain {chain}: {err}")

    # Split the FASTA text into individual lines.
    lines = response.text.strip().splitlines()

    current_header = None
    current_seq_lines = []

    # Process each line in the FASTA file.
    for line in lines:
        if line.startswith(">"):
            # If we have a previously accumulated record, check if it matches the desired chain.
            if current_header is not None and header_matches_chain(current_header, chain):
                return "".join(current_seq_lines).strip()
            # Begin a new record.
            current_header = line
            current_seq_lines = []
        else:
            # Accumulate sequence lines.
            current_seq_lines.append(line.strip())

    # Check the final record in the FASTA file.
    if current_header is not None and header_matches_chain(current_header, chain):
        return "".join(current_seq_lines).strip()

    # If no matching record is found, raise an error.
    raise ValueError(f"Chain '{chain}' not found in the FASTA for PDB {pdb_code}")


###############################################################################
# Function: header_matches_chain
###############################################################################
def header_matches_chain(header, chain):
    """
    Determine if the provided FASTA header contains the specified chain.

    The header is typically formatted with segments (e.g., "Chain A" or "Chains A, B[auth X]"). This
    function looks for such patterns and also checks for an "[auth ...]" designation if necessary.

    Parameters:
        header (str): The FASTA header string (without the leading '>').
        chain (str): The chain identifier to search for.

    Returns:
        bool: True if the chain is found in the header; otherwise, False.
    """
    # Remove the leading '>' if present.
    header = header.lstrip(">")
    # Split the header into parts using the '|' character as the delimiter.
    parts = header.split("|")

    # Examine each part for chain-related information.
    for part in parts:
        part_lower = part.lower()
        if part_lower.startswith("chain"):
            # Remove the prefix "chain" or "chains" (case-insensitive) and any bracketed annotations.
            cleaned = re.sub(r"(?i)chains?\s*", "", part)
            cleaned = re.sub(r"\[.*?\]", "", cleaned)
            # Split the cleaned string by commas or whitespace to get individual chain identifiers.
            chain_list = [c.strip() for c in re.split(r"[,\s]+", cleaned) if c.strip()]
            if chain in chain_list:
                return True

    # If the standard chain field is not found, search for an "[auth ...]" designation.
    auth_matches = re.findall(r"\[auth\s*([^\]]+)\]", header, flags=re.IGNORECASE)
    for auth in auth_matches:
        if auth.strip() == chain:
            return True

    return False


###############################################################################
# Function: extract_suffix
###############################################################################
def extract_suffix(filename):
    """
    Extract the binding class suffix from the filename by identifying which expected suffix it contains.

    The function examines the filename for one of the predefined suffixes and returns the suffix without
    the trailing ".txt". The expected suffixes include:
      - "500kNonMascotte.txt"
      - "superHeroesExclusive.txt"
      - "HeroesExclusive.txt"
      - "MascotteExclusive.txt"
      - "LooserExclusive.txt"

    Parameters:
        filename (str): The filename from which to extract the suffix.

    Returns:
        str or None: The extracted suffix (without ".txt") if found; otherwise, None.
    """
    # Define the list of possible suffixes (including the ".txt" extension).
    suffixes = [
        "500kNonMascotte.txt",
        "superHeroesExclusive.txt",
        "HeroesExclusive.txt",
        "MascotteExclusive.txt",
        "LooserExclusive.txt"
    ]

    # Iterate over the suffix list to see if the filename ends with one of the suffixes.
    for suffix in suffixes:
        if filename.endswith(suffix):
            # Remove the trailing ".txt" (last 4 characters) and return the suffix.
            return suffix[:-4]
    return None  # No matching suffix found.


###############################################################################
# Function: trim_antibody_from_sequence
###############################################################################
def trim_antibody_from_sequence(antibody_start, antibody_end, full_sequence):
    """
    Trim an antibody region from a full amino acid sequence based on user-specified start and end markers,
    returning only the antigen portion.

    The function assumes that the antibody region begins at the start of the sequence and extends until
    just before the provided antibody_end marker. A regular expression is dynamically constructed (with
    user-provided markers escaped) to identify the region.

    Parameters:
        antibody_start (str): The literal starting marker for the antibody region (e.g., "EVQL").
        antibody_end (str): The literal ending marker for the antibody region (e.g., "GSGT").
        full_sequence (str): The full amino acid sequence containing both the antibody and antigen regions.

    Returns:
        str: The antigen sequence with the antibody region removed. If the antibody region is not found,
             the full_sequence is returned unchanged.

    Example:
        >>> full_seq = "EVQLABCDEF123GSGTXYZ"
        >>> antigen = trim_antibody_from_sequence("EVQL", "GSGT", full_seq)
        >>> print(antigen)
        "XYZ"
    """
    # Escape the markers to ensure literal matching in the regex.
    start_esc = re.escape(antibody_start)
    end_esc = re.escape(antibody_end)

    # Build a regex pattern that:
    #   - Anchors to the start of the sequence (^)
    #   - Matches the antibody_start literal followed by any characters (non-greedy)
    #     until it reaches a position just before the antibody_end marker (using positive lookahead).
    pattern = r'^' + start_esc + r'(.*?)' + r'(?=' + end_esc + r')'

    match = re.search(pattern, full_sequence)
    if match:
        # The antibody region is the matched text starting at the beginning.
        antibody_region = match.group(0)
        # The antigen sequence is everything after the antibody region.
        antigen_sequence = full_sequence[len(antibody_region):]
        return antigen_sequence.strip()
    else:
        # If the antibody region is not found, return the full sequence.
        return full_sequence


###############################################################################
# Function: process_txt_files
###############################################################################
def process_txt_files(parent_dir):
    """
    Process all .txt files within the specified parent directory and its subdirectories.

    For each file, the function performs the following steps:
      1. Reads the first line to extract an antigen identifier in the format "#Antigen\tPDB_CHAIN".
      2. Parses the identifier to extract the PDB code and chain.
      3. Fetches the corresponding antigen sequence using the RCSB FASTA API.
      4. Uses the ANARCI tool to detect and annotate any antibody subsequence within the antigen sequence.
         If an antibody subsequence is detected, the file is flagged and (optionally) skipped.
      5. Extracts the binding class suffix from the filename.
      6. Reads the remainder of the file as a CSV (tab-delimited, with header starting on the second row).
      7. Augments the DataFrame with additional columns:
            - 'antigen': a concatenation of the PDB code and chain.
            - 'antigen_sequence': the fetched antigen sequence.
            - 'binding_class': the extracted suffix indicating the binding class.
      8. Saves the processed DataFrame as a CSV file in a "processed" subdirectory under the parent directory.

    If any error occurs during processing (e.g., HTTP errors, chain not found, CSV parsing errors), the file is
    skipped and processing continues with the next file.

    Parameters:
        parent_dir (str): The path to the parent directory containing the .txt files.

    Returns:
        list: A list of lists, where each inner list contains [pdb_code, chain, antigen_sequence] for every file
              that was successfully processed.
    """
    processed_results = []

    # Recursively walk through the parent directory.
    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            if file.endswith(".txt"):
                # Initialize flag to indicate if an antibody subsequence was detected.
                antigen_flag = False
                file_path = os.path.join(root, file)

                # Read the first line from the file to extract the antigen identifier.
                try:
                    with open(file_path, "r") as f:
                        first_line = f.readline().strip()
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
                    continue

                # Expected format is "#Antigen\tPDB_CHAIN" (e.g., "#Antigen\t4IJ3_A")
                parts = first_line.lstrip('#').split()
                if len(parts) < 2:
                    print(f"Unexpected format in {file_path}: {first_line}")
                    continue

                # Extract the PDB code and chain from the identifier.
                pdb_chain_str = parts[1]  # e.g., "4IJ3_A"
                try:
                    pdb_code, chain = pdb_chain_str.split('_')
                except ValueError:
                    # If the identifier does not follow the "PDB_CHAIN" format, use the entire string as pdb_code.
                    pdb_code = pdb_chain_str
                    chain = ""

                # Fetch the antigen sequence from the RCSB PDB.
                try:
                    seq = fetch_antigen_sequence(pdb_code, chain)
                except ValueError as e:
                    print(f"Error fetching sequence for {pdb_code} chain {chain} in file {file_path}: {e}")
                    continue

                # Record the successful processing result.
                processed_results.append([pdb_code, chain, seq])

                # Use ANARCI to detect antibody subsequences within the antigen sequence.
                # Prepare the input for ANARCI as a list of tuples: (name, sequence)
                name = f"{pdb_code}_{chain}"
                anarci_input = [(name, seq)]
                # Run ANARCI (if installed). Uncomment the next line if ANARCI is available.
                annotation = anarci.run_anarci(anarci_input, bit_score_threshold=120)
                hit = annotation[-2]
                if hit == [None]:
                    # No antibody subsequence detected.
                    pass
                else:
                    # If an antibody subsequence is detected, flag the file.
                    antibody_subsequence = annotation[0][0][1]
                    print(f"Antibody detected in: {name} {seq}")
                    antigen_flag = True
                    # The following lines (commented out) illustrate how one might trim the antibody region:
                    # start = ''.join([annotation[1][0][0][0][i][1] for i in range(0,6)])
                    # end = ''.join([annotation[1][0][0][0][i][1] for i in [-6,-5,-4,-3,-2,-1]])
                    # seq = trim_antibody_from_sequence(start, end, full_sequence=seq)
                    # print(f"Cleaned antibody out of sequence: {seq}")
                    # if len(seq) < 8:
                    #     print(f"Sequence too short after antibody removal: {seq}")
                    #     antigen_flag = True
                    #     continue
                    # else:
                    #     antigen_flag = False

                # Extract the binding class suffix from the filename.
                suffix_str = extract_suffix(filename=file)

                # Read the remainder of the file as a CSV (tab-delimited, header on the second line).
                try:
                    df = pd.read_csv(file_path, sep='\t', header=1)
                except Exception as e:
                    print(f"Error reading CSV from {file_path}: {e}")
                    continue

                # Augment the DataFrame with additional columns.
                antigen = f"{pdb_code}_{chain}"
                df['antigen'] = antigen
                df['antigen_sequence'] = seq
                df['binding_class'] = suffix_str

                # Ensure that a subdirectory called "processed" exists under the parent directory.
                processed_dir = os.path.join(parent_dir, 'processed')
                os.makedirs(processed_dir, exist_ok=True)
                # Build the output filename and file path.
                fname = f"{antigen}_{suffix_str}_processed.csv"
                fpath = os.path.join(processed_dir, fname)

                # Only save the processed DataFrame if no antibody subsequence was detected.
                if not antigen_flag:
                    try:
                        df.to_csv(fpath, index=False)
                        print(f"Saved processed file: {fpath}")
                    except Exception as e:
                        print(f"Error saving processed file {fpath}: {e}")
                        continue
                else:
                    print(f"Skipped writing because sequence is flagged as antibody: {fpath}")

    return processed_results

Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4IJ3_A_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/3VRL_C_500kNonMascotte_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4Hj0_A_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4RGM_S_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4YPG_C_HeroesExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/3R08_E_MascotteExclusive_proc

In [127]:
parent_directory = "/Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/"
process_txt_files(parent_directory)

Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4IJ3_A_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/3VRL_C_500kNonMascotte_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4Hj0_A_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4RGM_S_LooserExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/4YPG_C_HeroesExclusive_processed.csv
Saved processed file: /Volumes/LaCie_d2_Professional_Media/absolut_antibody/PerClass/RawBindingsPerClassMurine/extracted/processed/3R08_E_MascotteExclusive_proc

[['4IJ3',
  'A',
  'GSHMSALYSPSDPLTLLQADTVRGAVLGSRSAWAVEFFASWCGHCIAFAPTWKALAEDVKAWRPALYLAALDCAEETNSAVCRDFNIPGFPTVRFFKAFTKNGSGAVFPVAGADVQTLRERLIDALESHHDTWPPACPPLEPAKLEEIDGFFARNNEEYLALIFEKGGSYLGREVALDLSQHKGVAVRRVLNTEANVVRKFGVTDFPSCYLLFRNGSVSRVPVLMESRSFYTAYLQRLSGLTRE'],
 ['3VRL',
  'C',
  'PIVQNLQGQMVHQPISPRTLNAWVKVIEEKGFSPEVIPMFTALSEGATPQDLNTMLNTVGGHQAAMQMLKDTINEEAAEWDRLHPVQAGPVAPGQMRDPRGSDIAGTTSTLQEQIGWMTHNPPIPVGDIYKRWIILGLNKIVRMYSPVSILDIKQGPKESFRDYVDRFFKTLRAEQCTQDVKNWMTDTLLVQNANPDCKTILRALGPGATLEEMMTACQGVGGPSHKARVL'],
 ['4Hj0',
  'A',
  'MGSSHHHHHHSDYKDDDDKHMETGSKGQTAGELYQRWERYRRECQETLAAAEPPSGLACNGSFDMYVCWDYAAPNATARASCPWYLPWHHHVAAGFVLRQCGSDGQWGLWRDHTQCENPEKNEAFLDQRLILERLQ'],
 ['4RGM',
  'S',
  'GSEFGSESQPDPKPDELHKSSKFTGLMENMKVLYDDNHVSAINVKSIDQFLYFDLIYSIKDTKLGNYDNVRVEFKNKDLADKYKDKYVDVFGANYYYQCYFSKKTNDINSHQTDKRKTCMYGGVTEHNGNQLDKYRSITVRVFEDGKNLLSFDVQTNKKKVTAQELDYLTRHYLVKNKKLYEFNNSPYETGYIKFIENENSFWYDMMPAPGDKFDQSKYLMMYNDNKMVDSKDVKIEVYLTTKKK'],
 ['4YPG',
  'C',
  'TSCDLPQTHSLGSRRTLMLLAQMRKISL